In [20]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "5"
import random

import numpy as np
from elections.GaussianHelpers import cumulative_normal_dist

from elections.Candidate import Candidate
from ExperimentConfig import ExperimentConfig
from Experiment import Experiment
from CombinedExperiment import CombinedExperiment
from typing import List
from elections.PopulationGroup import Independents
from elections.Ideology import Ideology
from elections.HeadToHeadElection import HeadToHeadElection
from Experiment import RaceResult
from joblib import Parallel, delayed
from elections.GaussianHelpers import cumulative_normal_dist



x = ExperimentConfig("foo", "H2H",
    equal_pct_bins=True
)

def foo():
    for i in range(x.n_bins):
        p = x.convert_bin_to_ideology(i)
        print("%2d: p = % 7.3f  p_base = % 7.3f" % (i, p, x.pct_min + x.pct_step * i))

    for i in range(x.n_bins):
        ib_start = x.min_ideology + x.sigma_step * i
        ib_end = ib_start + x.sigma_step
        pct_start = cumulative_normal_dist(ib_start)
        pct_end = cumulative_normal_dist(ib_end)
        delta = pct_end - pct_start

        print("%2d ib_start % 6.3f pct_size %5.2f" % (i, ib_start, delta))

In [21]:

def compute_SUE(election_process: str, cv: float, flexibility: float):
    c = ExperimentConfig("v5",
                         election_process,
                         equal_pct_bins=False,
                         candidate_variance=cv,
                         ideology_flexibility = flexibility,
                         sampling_voters=1000,
                         model_path=f"exp/v5/{election_process}")


    x = Experiment(c)
    rr = x.run_strategic_races_par(1000)
    sue = CombinedExperiment.compute_SUE(rr)
    count_of_ties = 0
    for r in rr:
        if r.condorcet_tie:
            count_of_ties += 1
    ties = ""
    if election_process == "H2H":
        ties = "%6.2f" % (100 * count_of_ties / len(rr))


    print("%-10s flex %.2f cv %.2f SUE % 6.3f %s" % (election_process, flexibility, cv, sue, ties))

    return sue


# x = [1,2,3]
# y = [1,2,3]
#
# z = list(zip(x,y))
# print(z)
#
def make_plot_data(candidate_variance: float):
    flexibility = np.arange(0, 1.01, .1)
    plot_data = []
    processes  = ["IRV", "H2H"]
    print("computing processes")
    for process in processes:
           sue = [compute_SUE(process, .5, f) for f in flexibility]
           plot_data.append((flexibility, sue))

    return plot_data

In [33]:
class SUEComparison:
    def __init__(self, exp: Experiment, candidate_stddev: float, stddev_cap: float, span: bool):
        self.exp = exp
        self.stddev_cap = stddev_cap
        self.candidate_stddev = candidate_stddev
        self.span = span

    def gen_reference_candidates(self, n: int) -> List[Candidate]:
        candidates = []
        while len(candidates) < n:
            ivec = self.exp.config.population.unit_sample_voter().ideology.vec * self.exp.config.candidate_variance
            if abs(ivec[0]) < self.stddev_cap:
                quality = random.normalvariate(0, self.exp.config.quality_variance)
                candidates.append(Candidate(f"c-{len(candidates)}", Independents, Ideology(ivec), quality))
        ii = [c.ideology.vec[0] for c in candidates]
        min_ideology = np.min(ii)
        max_ideology = np.max(ii)
        if self.span and (min_ideology >  -.25 or max_ideology < .25):
            return self.gen_reference_candidates(n)
        return candidates

    def run_reference_election(self) -> RaceResult:
        candidates = self.gen_reference_candidates(5)
        voters = self.exp.config.population.generate_unit_voters(self.exp.config.sampling_voters)
        winner = self.exp.run_election(candidates, voters)
        utilities = self.exp.compute_utilities(winner, candidates, voters)
        return RaceResult(winner, candidates, candidates, utilities, utilities, HeadToHeadElection.count_of_ties > 0)

    def reference_results_par(self, n: int) -> List[RaceResult]:
        winners: List[RaceResult] = Parallel(n_jobs=16)(delayed(self.run_reference_election)() for _ in range(n))
        return winners

    def reference_results(self, n: int) -> List[RaceResult]:
        results = []
        for i in range(n):
            results.append(self.run_reference_election())

        return results

    def compute_reference_SUE(self) -> float:
        results = self.reference_results(100)
        sue = CombinedExperiment.compute_SUE(results)
        print("SUE for reference data is cv: %.2f cap %.2f span %6s % 6.2f" %
              (self.candidate_stddev, self.stddev_cap, self.span, sue))
        return sue


def run_one(cv: float, cap: float, span: bool):
    flexibility = 0
    election_process = "IRV"
    c = ExperimentConfig("v5",
                         election_process,
                         equal_pct_bins=False,
                         candidate_variance=cv,
                         ideology_flexibility = flexibility,
                         sampling_voters=200,
                         model_path=f"exp/v5/{election_process}")

    exp = Experiment(c)
    sue = SUEComparison(exp, cv, cap, span)
    sue.compute_reference_SUE()

def run_scan():
    for  span in [True, False]:
        for cv in [.5, 1.0]:
            for cap in [1.5, 2.0, 2.5, 20]:
                run_one(cv, cap, span)


run_scan()

SUE for reference data is cv: 0.50 cap 1.50 span   True   0.46
SUE for reference data is cv: 0.50 cap 2.00 span   True   0.42
SUE for reference data is cv: 0.50 cap 2.50 span   True   0.45
SUE for reference data is cv: 0.50 cap 20.00 span   True   0.44
SUE for reference data is cv: 1.00 cap 1.50 span   True   0.84
SUE for reference data is cv: 1.00 cap 2.00 span   True   0.86
SUE for reference data is cv: 1.00 cap 2.50 span   True   0.88
SUE for reference data is cv: 1.00 cap 20.00 span   True   0.91
SUE for reference data is cv: 0.50 cap 1.50 span  False   0.56
SUE for reference data is cv: 0.50 cap 2.00 span  False   0.63
SUE for reference data is cv: 0.50 cap 2.50 span  False   0.56
SUE for reference data is cv: 0.50 cap 20.00 span  False   0.54
SUE for reference data is cv: 1.00 cap 1.50 span  False   0.80
SUE for reference data is cv: 1.00 cap 2.00 span  False   0.90
SUE for reference data is cv: 1.00 cap 2.50 span  False   0.90
SUE for reference data is cv: 1.00 cap 20.00 span  F

In [31]:
cumulative_normal_dist(1.5)


0.9331927712798016

In [ ]:
import matplotlib.pyplot as plt
def make_line_plot(data, title, labels):
    n_rows = 1
    n_cols = 1
    fig, axis = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(20, 10))
    fig.suptitle(title, color="black", fontsize=22)
    fig.set_facecolor("white")

    count = 0
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    axis.tick_params(axis='x', colors="black")

    for i in range(len(data)):
        d = data[i]
        plt.plot(d[0], d[1], label =labels[i])
    plt.legend()

    axis.set_xlabel("Candidate Ideological Flexibility (stddev)", fontsize=20)
    axis.set_ylabel("Social Utility Efficiency", fontsize=20)

    plt.savefig("foo.png")


plot_data_05 = make_plot_data(0.5)
plot_data_10 = make_plot_data(1.0)
make_line_plot(plot_data_05, "Social Utility Efficiency vs. Flexibility for Candidate Variance 0.5", ["Instant Runoff", "Condorcet-Minimax"])
make_line_plot(plot_data_10, "Social Utility Efficiency vs. Flexibility for Candidate Variance 1.0", ["Instant Runoff", "Condorcet-Minimax"])

computing processes
IRV        flex 0.00 cv 0.50 SUE  0.340 
IRV        flex 0.10 cv 0.50 SUE  0.364 
IRV        flex 0.20 cv 0.50 SUE  0.319 
IRV        flex 0.30 cv 0.50 SUE  0.309 
IRV        flex 0.40 cv 0.50 SUE  0.254 
IRV        flex 0.50 cv 0.50 SUE  0.305 
IRV        flex 0.60 cv 0.50 SUE  0.354 
IRV        flex 0.70 cv 0.50 SUE  0.379 
IRV        flex 0.80 cv 0.50 SUE  0.419 
IRV        flex 0.90 cv 0.50 SUE  0.442 
IRV        flex 1.00 cv 0.50 SUE  0.491 
H2H        flex 0.00 cv 0.50 SUE  0.991   1.00
H2H        flex 0.10 cv 0.50 SUE  1.000   1.00
H2H        flex 0.20 cv 0.50 SUE  1.023   2.40
H2H        flex 0.30 cv 0.50 SUE  1.049   5.40
H2H        flex 0.40 cv 0.50 SUE  1.067   7.20
H2H        flex 0.50 cv 0.50 SUE  1.084   7.30
H2H        flex 0.60 cv 0.50 SUE  1.081  11.90
H2H        flex 0.70 cv 0.50 SUE  1.096  12.90
H2H        flex 0.80 cv 0.50 SUE  1.094  17.00
H2H        flex 0.90 cv 0.50 SUE  1.103  17.80
H2H        flex 1.00 cv 0.50 SUE  1.101  18.20
computing pr